# Introduction to Data Science

Before you hand this problem in, make sure everything runs as expected. You should **restart the kernel and run all cells** by selecting 

`Kernel --> Restart Kernel and Run All Cells`

in the menubar.

- Of course, you should use **an appropriate kernel** on the Jupyterhub of the math department or locally, so that the right modules are used and the calculations can be checked deterministically.  
- Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".
- Rename this problem sheet as follows:

      ps{number of lab}_{your user name}_problem{number of problem sheet in this lab}
    
  for example
    
      ps02_blja_problem1
    
- Please fill out the cell below for **every submission**.

**Change in submission of files**: Please upload this submission until next Tuesday to your shared Nextcloud [https://tuc.cloud/](https://tuc.cloud/) directory with the name of your username which has been created during the third exercise lab.
If you have not yet been assigned to a shared Nextcloud folder, please contact me via email (jan.blechschmidt@mathematik.tu-chemnitz.de) as soon as possible.

In [28]:
NAME = "Tanay Maurya"
EMAIL = "tanay.maurya@s.2024.tu-chemnitz.de"
USERNAME = "tanay"

---

# Introduction to Data Science
## Lab 6: Test for importance of a subset of predictors

In this problem, we want to investigate another data set.
The data come from a study of [Stamey et al. (1989)](https://www.sciencedirect.com/science/article/pii/S002253471741175X?via%3Dihub).
It consists of some clinical measurements of 97 patients, who where about to receive a prostatectomy.

The predictors are:
* lcavol - Logarithm of cancer volume
* lweight - Logarithm of prostate weight
* age - Age of patient
* lbph - Logarithm of amount of benign prostatic hyperplasia
* svi - Seminal vesicle invasion
* lcp - Logarithm of capsular penetration
* gleason - Gleason score
* pgg45 - Percent of Gleason scores 4 or 5

The variable that we want to predict is:
* lpsa - Level of prostate-specific antigen

There are no missing values.

The first task is to download and import the `prostate` data set.

Taking a short look at the data reveals that the data is seperated by tabs.
You can use the convenient `pandas` method `read_csv` with the options `sep = "\t"` and `index_col = 0`.

In [29]:
import pandas as pd
import numpy as np
# Import prostate cancer dataset
pc = pd.read_csv('prostate.data', sep="\t", index_col=0)

The method `head` of a `pd.DataFrame` prints by default the first 5 rows of the `DataFrame`, which is suitable for getting an overview of the data.

**Task**: Try it!

In [30]:
# YOUR CODE HERE
pc.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


As you can see, there is a column named **train**, which is either `"T"` (True) or `"F"` (False).
This means that the data has already been split up into a training and a test set, which we will use in some minutes.

**Task**: You should also take a look at the correlation matrix using the method `corr`.
What do you observe, especially concerning the correlation between the features? It might help to colorize the visualization of the correlation matrix using the method `style.background_gradient` of `pandas.DataFrame`s.

In [31]:
# YOUR CODE HERE
pc_corr = pc.drop(columns=["train"]).corr()
pc_corr.style.background_gradient(cmap='Blues')

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
lcavol,1.000000,0.280521,0.225000,0.027350,0.538845,0.675310,0.432417,0.433652,0.734460
lweight,0.280521,1.000000,0.347969,0.442264,0.155385,0.164537,0.056882,0.107354,0.433319
age,0.225000,0.347969,1.000000,0.350186,0.117658,0.127668,0.268892,0.276112,0.169593
lbph,0.027350,0.442264,0.350186,1.000000,-0.085843,-0.006999,0.077820,0.078460,0.179809
svi,0.538845,0.155385,0.117658,-0.085843,1.000000,0.673111,0.320412,0.457648,0.566218
lcp,0.675310,0.164537,0.127668,-0.006999,0.673111,1.000000,0.514830,0.631528,0.548813
gleason,0.432417,0.056882,0.268892,0.077820,0.320412,0.514830,1.000000,0.751905,0.368987
pgg45,0.433652,0.107354,0.276112,0.078460,0.457648,0.631528,0.751905,1.000000,0.422316
lpsa,0.734460,0.433319,0.169593,0.179809,0.566218,0.548813,0.368987,0.422316,1.000000


You should observe that there are many predictor variables that have a high correlation.
High correlation is always an *indicator* of multiple features having similar effects on the independent variable.

From now on, we concentrate on the numerical values of the data set:
- extract the predictors of the dataset as a `numpy.array` $X$
- extract the column **lspa** as a `numpy.array` $y$

In [32]:
# YOUR CODE HERE
predictors = ["lcavol", "lweight", "age", "lbph", "svi", "lcp", "gleason", "pgg45"]
X = pc[predictors].to_numpy()
y = pc["lpsa"].to_numpy()

In [33]:
assert X.shape == (97,8)
assert y.shape == (97,)

# This cell contains further tests, don't remove!

Before we further analyze our data, we want to normalize the predictor variables.
This technique is often necessary when comparing different kinds of inputs.

Here, we want to normalize the **predictor variables** such that the *normalized predictor variables* have mean $0$ and variance $1$.

**Caution**: There are a lot of functions that sound like they're doing the right thing, but there are a lot more ways to normalize a data set, e.g. $l^2$-normalization etc.

**Task**: Normalize the predictor variables, you can use the function `scale` from `scikit-learn`:

    from sklearn.preprocessing import scale

You should name the normalized variable `Xnorm`, otherwise the following code might not work.
Print the mean value of each predictor as well as the varianze and make sure that they are zero and one, resp.
You can use the function `np.array2string` to apply formatting options to each entry of a `numpy.array` and beautify and improve the readability of a printed array.

In [34]:
# YOUR CODE HERE
from sklearn.preprocessing import scale
Xnorm = scale(X)
mean = np.array2string(Xnorm.mean(), formatter={'float_kind': lambda x: f"{0 if abs(x) < 1e-10 else x:.4f}"})
variance = np.array2string(Xnorm.var())

print("Means of normalized predictors:", mean)
print("Variances of normalized predictors:", variance)

Means of normalized predictors: 0.0000
Variances of normalized predictors: 1.


In [35]:
assert max(abs(Xnorm.mean(axis=0))) < 1e-10
assert max(abs(Xnorm.var(axis=0)-1)) < 1e-10

The next step is to divide the data set into a training and a test data set.
Here, the assignment of training and test data has already been done.
You can extract the column **train** by

    is_train = (pc["train"] == "T")
    
which is simply a `pandas.Series` object containing the values `True` and `False`.
This object contains boolean values, i.e., `True` and `False` values.
We can use this *filter* to get our training data by

    Xtrain = Xnorm[is_train, :]
    
The same indexing can be applied to $y$.

**Task**: Devide the dataset into training and test samples and save them under `Xtrain`, `ytrain`, `Xtest`, `ytest`.

In [36]:
# YOUR CODE HERE
is_train = (pc["train"] == "T")
Xtrain = Xnorm[is_train, :]
ytrain = y[is_train]        

Xtest = Xnorm[~is_train, :]  
ytest = y[~is_train]         

In [37]:
assert Xtrain.shape == (67,8)
assert ytrain.shape == (67,)

# This cell contains further tests, don't remove!

Now, we want to fit a linear regression model on our **training set** using all of the predictor variables.
In the last tutorials, we have employed `numpy`s class `numpy.polynomial.polynomial.Polynomial`.
Here, you should use the following class:

    from sklearn.linear_model import LinearRegression
    
Have a look at its documentation and fit the model.
Store the trained intercept in the variable `intercept` as well as the regression coefficients (as a list) in the variable `coeffs`.

In [38]:
# YOUR CODE HERE
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(Xtrain, ytrain)


intercept = model.intercept_ 
coeffs = model.coef_.tolist()  

# Verify the results
print(f"Intercept: {intercept}")
print(f"Coefficients: {coeffs}")

Intercept: 2.4649329221237446
Coefficients: [0.6760163443892511, 0.26169360926381857, -0.14073374423100682, 0.2090605212320933, 0.3036233224985313, -0.2870018437521441, -0.021194934506172228, 0.26557613654260215]


In [39]:
# This cell contains further tests, don't remove!

Download the file `multipleTTest.py` which contains basically the function that we used in problem sheet 04.
After downloading the file and moving it into the current directory, you can import it using
    
    from multipleTTest import multipleTTest

This function enhances the capability of our previous method `computeTStatistic` slightly:
* You may now also provide a `labels` object that contains the names of the predictor variables and modifies only the output table. You can get the labels of a `pandas.DataFrame` by
    
        labels = pc.keys()
     
* Additionally, the function `multipleTTest` returns the residual sum of squares of the linear regression fit. 
* You may set the optional input variable `includeIntercept` to `True`. This appends the intercept in the estimate, and you do not have to put a column containing ones by yourself.

**Task**: Execute the function for the normalized training data set. Store the RSS value as `RSS1`. Store also the number of variables in this model as `p1` (count the intercept as one variable!).
How many variables are not significant at a threshold of 5 %?
Store your answer in the variable `notSign1`.

In [45]:
# YOUR CODE HERE
from multipleTTest import multipleTTest
from scipy.stats import t
labels = pc.keys()
RSS1 = multipleTTest(Xtrain, ytrain, includeIntercept = True, labels=labels)
p1 = Xtrain.shape[1]+1
def get_p_values(X, y):
    n, p = X.shape
    X_with_intercept = np.hstack((np.ones((n, 1)), X))  # Add intercept
    V = np.linalg.inv((X_with_intercept.T).dot(X_with_intercept))
    beta = V.dot(X_with_intercept.T.dot(y))
    y_pred = X_with_intercept.dot(beta)
    RSS = np.sum((y - y_pred) ** 2)
    sigma_hat = np.sqrt(RSS / (n - p - 1))
    SE = np.sqrt(V.diagonal()) * sigma_hat
    t_vals = beta / SE
    p_vals = 2 * (1 - t.cdf(np.abs(t_vals), n - p - 1))
    return p_vals
p_values = get_p_values(Xtrain, ytrain)

# Count the number of non-significant variables (p-value > 0.05)
notSign1 = np.sum(p_values > 0.05)

# Print the results
print(f"Residual Sum of Squares (RSS1): {RSS1}")
print(f"Number of variables in the model (p1): {p1}")
print(f"Number of non-significant variables (notSign1): {notSign1}")

|  Coefficient | Estimate |    SE    | t-statistic |  p-value  |
---------------------------------------------------------------
|  Intercept   |   2.465  |  0.0893  |    27.60    | < 0.0001  |
|     lcavol   |   0.676  |  0.1260  |     5.37    | < 0.0001  |
|    lweight   |   0.262  |  0.0951  |     2.75    |   0.0079  |
|        age   |  -0.141  |  0.1008  |    -1.40    |   0.1681  |
|       lbph   |   0.209  |  0.1017  |     2.06    |   0.0443  |
|        svi   |   0.304  |  0.1230  |     2.47    |   0.0165  |
|        lcp   |  -0.287  |  0.1537  |    -1.87    |   0.0670  |
|    gleason   |  -0.021  |  0.1445  |    -0.15    |   0.8839  |
|      pgg45   |   0.266  |  0.1528  |     1.74    |   0.0875  |
Residual Sum of Squares (RSS1): 29.4263844599084
Number of variables in the model (p1): 9
Number of non-significant variables (notSign1): 4


In [46]:
# This cell contains further tests, don't remove!

We want to compare the fit containing all predictor variables against a model containing only a subset of variables.
The variables that we want to exclude are `age`, `lcp`, `gleason` and `pgg45`.

**Task**: Use the function `multipleTTest` to perform a test using only this subset of variables.
Store the residual sum of squares as `RSS0` and the number of variables as `p0`.

In [50]:
# YOUR CODE HERE
# Define the variables to exclude
exclude_vars = ['age', 'lcp', 'gleason', 'pgg45']

# Select only the subset of variables (excluding the specified variables)
subset_vars = [col for col in pc.columns[:-1] if col not in exclude_vars]
X_subset = pc[subset_vars].to_numpy()

# Split the data into training set for the subset of variables
Xtrain_subset = X_subset[is_train, :]  # Subset training predictors

# Run the multipleTTest function for the subset of variables
RSS0 = multipleTTest(Xtrain_subset, ytrain, includeIntercept=True, labels=subset_vars)

# Store the number of variables in the subset model (including intercept)
p0 = Xtrain_subset.shape[1] + 1  # Number of predictors in the subset + 1 for intercept

# Print the results
print(f"Residual Sum of Squares (RSS0): {RSS0}")
print(f"Number of variables in the subset model (p0): {p0}")

|  Coefficient | Estimate |    SE    | t-statistic |  p-value  |
---------------------------------------------------------------
|  Intercept   |   0.000  |  0.0000  |     1.37    |   0.1745  |
|     lcavol   |   0.000  |  0.0000  |     0.26    |   0.7977  |
|    lweight   |  -0.000  |  0.0000  |    -0.50    |   0.6174  |
|       lbph   |   0.000  |  0.0000  |     0.20    |   0.8424  |
|        svi   |   0.000  |  0.0000  |     0.00    |   1.0000  |
|       lpsa   |   1.000  |  0.0000  |    257142729870932.06    | < 0.0001  |
Residual Sum of Squares (RSS0): 3.027287543115456e-26
Number of variables in the subset model (p0): 6


In [ ]:
# This cell contains further tests, don't remove!

According to the lecture, we may now test the reduced model against the full model using an appropriate $F$-statistic.
Thus, we test the null hypothesis:

$$ \textbf{H}_0: \beta_{\text{age}} = \beta_{\text{lcp}} = \beta_{\text{gleason}} = \beta_{\text{pgg45}} = 0$$

against

$$ \textbf{H}_1: \text{at least one of the variables } \beta_{\text{age}}, \beta_{\text{lcp}}, \beta_{\text{gleason}}, \beta_{\text{pgg45}} \text{ is not zero } $$

We can do this by computing the $F$-stastitic:

$$ F = \frac{(RSS_0 - RSS_1) / (p_1 - p_0)}{RSS_1 / (n - p_1)} $$

while $n$ is the number of training samples.

According to our assumptions, $F$ will have an $F$-distribution with $(p_1-p_0, n-p_1)$ degrees of freedom.

**Task**: Compute the value of the test statistic and store it in a variable `F`.

In [51]:
# YOUR CODE HERE
# Compute the number of training samples
n = Xtrain.shape[0]

# Compute the F-statistic
F = ((RSS0 - RSS1) / (p1 - p0)) / (RSS1 / (n - p1))

# Print the F-statistic
print(f"F-statistic: {F}")


F-statistic: -19.333333333333332


In [52]:
assert 'F' in locals()

# This cell contains further tests, don't remove!

To compute the $p$-value, we may import the $F$-distribution by

    from scipy.stats import f as f_dist
    
**Task**: Determine the corresponding $p$-value and store the value in a variable `pval`.
Assuming a level of significance of 5%, can we safely reject the null hypothesis? Store either `True` or `False` in the variable `reject_null`.

In [53]:
# YOUR CODE HERE
from scipy.stats import f as f_dist

# Compute the p-value for the F-statistic
df1 = p1 - p0  # Degrees of freedom for the numerator (difference in model complexity)
df2 = n - p1   # Degrees of freedom for the denominator (full model)

# Calculate the p-value
pval = 1 - f_dist.cdf(F, df1, df2)

# Check if we can reject the null hypothesis at the 5% significance level
reject_null = pval < 0.05

# Print the results
print(f"p-value: {pval}")
print(f"Reject null hypothesis (at 5% level): {reject_null}")

p-value: 1.0
Reject null hypothesis (at 5% level): False


In [54]:
assert 'pval' in locals()
assert 'reject_null' in locals()

# This cell contains further tests, don't remove!

**Task**: Finally, you should train and evaluate the full model and the reduced model.
Compare both in terms of the $R^2$-value on the test set.
Which model performs better on the training set and the test set, resp.?
Try to explain your findings and store your answer in a comment.

In [59]:
# YOUR CODE HERE


IndexError: index 8 is out of bounds for axis 1 with size 8